## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import requests
from bs4 import BeautifulSoup
from shapely.geometry import Point

## Read-in

In [16]:
df = pd.read_csv('LISF_Mar_Apr_2024.csv',dtype='str')

In [20]:
df['Doc Recorded'].isna().value_counts()

Doc Recorded
False    3338
Name: count, dtype: int64

In [130]:
631 + 713

1344

In [37]:
len(df[df['Doc Recorded'].str.startswith('3')])

1744

In [38]:
# Make sure to update this
print('Enter report month as numeric. Example: January is 1, February is 2, etc...')
first_month = int(input())
print('Enter report month as named string. Example: January...February...')
current_month = input()

Enter report month as numeric. Example: January is 1, February is 2, etc...
3
Enter report month as named string. Example: January...February...
March


In [39]:
print(first_month)
print('---------')
print(current_month)

3
---------
March


In [40]:
len(df)

3338

In [41]:
# Define the regex pattern to split the text
pattern = r'(\d+-\d+-\d+-\d+-\d+)\s(.*)'

# Apply regex and split the text into two columns
df[['PIN', 'Address']] = df['1st PIN'].str.extract(pattern)

# Remove leading/trailing whitespace from the address column
df['Address'] = df['Address'].str.strip()

In [42]:
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0','1st PIN'])

## Clean, drop, and convert

In [43]:
df['Doc Recorded'] = pd.to_datetime(df['Doc Recorded'], format='mixed')

In [44]:
df = df.drop_duplicates()

In [45]:
len(df)

2438

## Data Stuff

In [10]:
# # Count number of earliest month dates
# sep_dates_count = len(df.loc[df['Doc Recorded'].dt.month == first_month])

# print(f'Number of September dates: {sep_dates_count}')

In [46]:
df['1st Grantor'] = df['1st Grantor'].fillna('NA')
df['1st Grantee'] = df['1st Grantee'].fillna('NA')

In [47]:
municipal_authority = ['city', 'town', 'municipality', 'village','transit auth','department of transp']  # list of municipal authority keywords
df = df[~df['1st Grantor'].str.contains('|'.join(municipal_authority), case=False) & 
        ~df['1st Grantee'].str.contains('|'.join(municipal_authority), case=False)]

In [48]:
# create new column and assign colors based on month
df['COLOR'] = df['Doc Recorded'].apply(lambda x: 'red' if x.month == first_month else '')

## Get mortgage amounts

In [49]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers'
}

In [50]:
def mortgage_url_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    for link in soup.find_all('a', href=True):
        if link['href'].startswith('/Document/Detail'):
            mortgage_url = 'https://crs.cookcountyclerkil.gov' + link['href']
            return mortgage_url

In [51]:
def mortgage_consi_snagger(URL, headers):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find all tr tags
    trs = soup.find_all('tr')

    # Loop through each tr tag and look for the td tag containing 'Consideration Amount' label
    for tr in trs:
        td = tr.find('th', text='Consideration Amount:')
        if td:
            # If the td tag is found, get the next td tag containing the amount
            amount_td = td.find_next_sibling('td')
            if amount_td:
                # Print the amount
                amount = amount_td.text.strip()
                return amount
            else:
                return 'not found'

In [52]:
df['mortgage_urls'] = df['deed_urls'].apply(lambda x: mortgage_url_snagger(x, headers))

In [54]:
df['mortgage_amount'] = df['mortgage_urls'].apply(lambda x: mortgage_consi_snagger(x, headers) if x is not None else None)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_20181/2390556658.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  td = tr.find('th', text='Consideration Amount:')


In [55]:
df[['mortgage_urls','mortgage_amount']] = df[['mortgage_urls','mortgage_amount']].fillna('NA')

## Geocode

In [56]:
len(df)

2067

In [58]:
df['geo_address'] = df['Address'] + ' Cook County, IL'

In [59]:
len(df)

2067

In [60]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [61]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['geo_address'].apply(geocode)

In [62]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [63]:
df.columns

Index(['View Doc', 'Doc Number', 'Doc Recorded', 'Doc Executed', 'Doc Type',
       'Consi. Amt.', '1st Grantor', '1st Grantee', 'Assoc. Doc#', 'deed_urls',
       'PIN', 'Address', 'COLOR', 'mortgage_urls', 'mortgage_amount',
       'geo_address', 'geocoded', 'lat', 'lon'],
      dtype='object')

In [64]:
def popup_html(row):
    grantor = row['1st Grantor']
    grantee = row['1st Grantee']
    PIN = row['PIN']
    Address = row['Address']
    mortgage_amount = row['mortgage_amount']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Lender: </strong>{}'''.format(grantor) + '''<br>
    <strong>Borrower: </strong>{}'''.format(grantee) + '''<br>
    <strong>PIN: </strong>{}'''.format(PIN) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>Mortgage Amount: </strong>{}'''.format(mortgage_amount) + '''<br>
    </html>
    '''
    return html

In [78]:
df['COLOR'].value_counts()

COLOR
red       1090
yellow     977
Name: count, dtype: int64

In [71]:
# df.to_csv("backup_copy.csv")

In [76]:
df['COLOR'] = df['COLOR'].replace('', 'yellow')

In [82]:
import folium
from folium.plugins import MarkerCluster
import numpy as np

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Cook County Pending Foreclosures')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{current_month} and April')

### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.5, tiles=None)

# Create two FeatureGroups for different color pins
fg_red = folium.FeatureGroup(name=f'{current_month}') ## 
fg_yellow = folium.FeatureGroup(name='April')

for index, row in df.iterrows():
    lat = row['lat']
    lon = row['lon']
    color = row['COLOR']
    if pd.notnull(lat) and pd.notnull(lon) and color == 'red':
        marker = folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            fill=True,
            color=color,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_red)
    elif pd.notnull(lat) and pd.notnull(lon) and color == 'yellow':
        marker = folium.CircleMarker(
            location=[lat, lon],
            radius=5,
            fill=True,
            color=color,
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_yellow)
    else:
        continue

# Add the FeatureGroups to the map
fg_red.add_to(m)
fg_yellow.add_to(m)

folium.TileLayer('OpenStreetMap', control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
folium.TileLayer('CartoDBpositron', control=False).add_to(m) 
            
# Display map
m

In [83]:
m.save('index.html')

## Map URL Snagger

In [84]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/lis_pendens_scraper_march_2024


## Stats for story

In [85]:
df['mortgage_amount_int'] = df['mortgage_amount'].str.replace('$','')
df['mortgage_amount_int'] = df['mortgage_amount_int'].str.replace(',','')
df = df.loc[df['mortgage_amount_int'] != 'NA']
df['mortgage_amount_int'] = df['mortgage_amount_int'].astype(float).fillna(0).astype(int)

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_20181/159485817.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mortgage_amount_int'] = df['mortgage_amount_int'].astype(float).fillna(0).astype(int)


In [86]:
# Count number of first month dates
dates_count = len(df.loc[df['Doc Recorded'].dt.month == first_month])

print(f'Number of {current_month} dates: {dates_count}')

Number of March dates: 1030


In [87]:
df['Doc Recorded'].dt.month.value_counts()

Doc Recorded
3    1030
4     909
Name: count, dtype: int64

In [96]:
# Filter for March
df_march = df[df['Doc Recorded'].dt.month == 3]

# Filter for April
df_april = df[df['Doc Recorded'].dt.month == 4]

In [99]:
df_march.reset_index(inplace=True, drop=True)
df_april.reset_index(inplace=True, drop=True)

In [36]:
# x.at[123,'mortgage_amount_int']=33130000
# x.at[123,'mortgage_amount']='$33,130,000'

# x.at[16,'mortgage_amount_int']=173600
# x.at[16,'mortgage_amount']='$173,600'

In [122]:
print(f'March foreclosures: {len(df_march)}')
print('----------')
print(f'April foreclosures: {len(df_april)}')

March foreclosures: 1030
----------
April foreclosures: 909


In [100]:
df_march.sort_values(by='mortgage_amount_int',ascending=False).iloc[0:1]

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
142,View,2408114068,2024-03-21,3/20/2024,LIS PENDENS FORECLOSURE,NaN,WILMINGTON TRUST NATIONAL ASSN TR,A&M 205 RANDOLPH LLC,1500929050.0,https://crs.cookcountyclerkil.gov/Document/Det...,17-09-443-006-0000,"209 W RANDOLPH ST, CHICAGO",red,https://crs.cookcountyclerkil.gov/Document/Det...,"$18,700,000.00","209 W RANDOLPH ST, CHICAGO Cook County, IL","(41.8842314, -87.6342593)",41.884231,-87.634259,18700000


In [132]:
print(df_march.iloc[142]['mortgage_urls'])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=MzM4OTgxNDg1&hId=MDVlMmVmYWQxODZjOTFiNjhlMDRkMDhmZDJlZjY1MDZhOTUyYWU0ODFlNWE5MzM5OGZlNmIwYjIyMDc0Y2NlYg2


In [133]:
df_april.sort_values(by='mortgage_amount_int',ascending=False).iloc[0:1]

,View Doc,Doc Number,Doc Recorded,Doc Executed,Doc Type,Consi. Amt.,1st Grantor,1st Grantee,Assoc. Doc#,deed_urls,PIN,Address,COLOR,mortgage_urls,mortgage_amount,geo_address,geocoded,lat,lon,mortgage_amount_int
111,View,2410914185,2024-04-18,NaN,LIS PENDENS FORECLOSURE,NaN,PARKWAY BK AND TRUST COMPANY,LV HALSTED LLC,2301313309.0,https://crs.cookcountyclerkil.gov/Document/Det...,14-33-313-027-0000,"1631 N HALSTED ST, CHICAGO",yellow,https://crs.cookcountyclerkil.gov/Document/Det...,"$4,997,407.00","1631 N HALSTED ST, CHICAGO Cook County, IL","(41.91186099999999, -87.6479969)",41.911861,-87.647997,4997407


In [107]:
print(df_april['mortgage_urls'].iloc[111])

https://crs.cookcountyclerkil.gov/Document/Detail?dId=NDQyNzY4Mjc1&hId=ZDE3NjAwMTVlMWNkZDQ4YzQ5ZDdiZmUwNzQ3MGFiNDczMGUyMTA1OWMzN2U5Y2Q1N2Q4Njk0YmE0YzQ3ZTlkMg2


In [108]:
print(f'{current_month}: ${df_march.mortgage_amount_int.sum():,}')

March: $236,908,070


In [110]:
print(f'April: ${df_april.mortgage_amount_int.sum():,}')

April: $211,358,482


In [134]:
80 + 36 + 33 + 14

163

In [111]:
df_march['1st Grantor'].value_counts().head(60)

1st Grantor
US BK NATIONAL ASSOCIATION                                80
FREEDOM MTG CORPORATION                                   70
NATIONSTAR MTG LLC                                        59
DEUTSCHE BK NATIONAL TRUST COMPANY TR                     52
WELLS FARGO BK NA                                         51
LAKEVIEW LOAN SERVICING LLC                               51
WILMINGTON SAV FUND SOCIETY FSB TR                        40
US BANK NATIONAL ASSN TR                                  36
FIFTH THIRD BK NATIONAL ASSOCIATION                       35
US BANK TRUST NATIONAL ASSN TR                            33
PENNYMAC LOAN SERVICES LLC                                33
BANK OF AMER NA                                           32
NEWREZ LLC                                                25
LOANDEPOTCOM LLC                                          25
CARRINGTON MTG SERVICES LLC                               20
PNC BK NATIONAL ASSOCIATION                               20
BANK OF NEW 

In [135]:
54 + 50 + 26 + 11 + 4

145

In [112]:
df_april['1st Grantor'].value_counts().head(60)

1st Grantor
JPMORGAN CHASE BK NATIONAL ASSOCIATION    62
US BK NATIONAL ASSOCIATION                54
LAKEVIEW LOAN SERVICING LLC               52
US BANK TRUST NATIONAL ASSN TR            50
DEUTSCHE BK NATIONAL TRUST COMPANY TR     45
FREEDOM MTG CORPORATION                   37
NATIONSTAR MTG LLC                        32
NEWREZ LLC                                30
CARRINGTON MTG SERVICES LLC               29
US BANK NATIONAL ASSN TR                  26
WILMINGTON SAV FUND SOCIETY FSB TR        25
PENNYMAC LOAN SERVICES LLC                24
MIDFIRST BANK                             21
WELLS FARGO BK NA                         19
ROCKET MTG LLC                            19
CITIZENS BK NA                            14
PNC BK NATIONAL ASSOCIATION               13
PHH MTG CORPORATION                       13
LOANDEPOTCOM LLC                          13
BANK OF AMER NA                           12
FIFTH THIRD BK NATIONAL ASSOCIATION       12
US BANK TRUST COMPANY NATIONAL ASSN TR    1

In [113]:
df_march['1st Grantee'].value_counts()

1st Grantee
CHICAGO TITLE LAND TRUST CO TR    4
HUSSAIN ALI                       2
PORTER JASON                      2
OLSHANSKY EDWARD                  2
ILLINOIS HOUSING DEV AUTHORITY    2
                                 ..
SLAY JAMAAR                       1
FERNANDEZ IRENE A                 1
PARKER EFEREM                     1
DIXON TAMIRRA                     1
RAZZAQ ALI                        1
Name: count, Length: 1020, dtype: int64

In [114]:
df_april['1st Grantee'].value_counts()

1st Grantee
ALJ INVESTMENTS INC               5
JACKSON ANDRE L                   5
LV HALSTED LLC                    4
JACKSON ANDRE                     4
US BANK TRUST NATIONAL ASSN TR    2
                                 ..
FLETCHER DOUGLAS G JR             1
HAMLET DEQUINCY L                 1
RELIFORD BILLY G JR               1
MEDINA ROCIO                      1
RUPERT CURTIS                     1
Name: count, Length: 886, dtype: int64

In [95]:
df.to_csv(f'{current_month}_April_2024_foreclosures.csv')

In [44]:
# apr.sort_values(by='mortgage_amount_int',ascending=False)

In [115]:
print(f"{current_month} median mortgage foreclosure amount: ${df_march['mortgage_amount_int'].median():,}")

March median mortgage foreclosure amount: $174,387.5


In [116]:
print(f"April median mortgage foreclosure amount: ${df_april['mortgage_amount_int'].median():,}")

April median mortgage foreclosure amount: $168,750.0


## Boundary Analysis

In [117]:
boundaries = gpd.read_file('Boundaries - Neighborhoods.geojson')

In [118]:
# create Point objects from lat/lon columns in df
geometry = gpd.points_from_xy(x['lon'], x['lat'])

# create geodataframe from df with Point objects as geometry
gdf_filings_x = gpd.GeoDataFrame(x, geometry=geometry)

gdf_filings_x.crs = 'EPSG:4326'
gdf_filings_x = gdf_filings_x.to_crs(boundaries.crs)

# use contains method to get count of points within each neighborhood
counts_x = gpd.sjoin(gdf_filings_x, boundaries, predicate='within').groupby('pri_neigh').size().reset_index(name='count')

In [119]:
counts_x['count'].sum()

517